<a href="https://colab.research.google.com/github/mbhupe/VeracrossSchedule/blob/main/davis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install reportlab
!pip install gdown
import csv
import os
import gdown
import re
from reportlab.lib.enums import TA_RIGHT
from reportlab.lib.styles import ParagraphStyle
from reportlab.lib.enums import TA_LEFT, TA_CENTER, TA_RIGHT, TA_JUSTIFY

from reportlab.lib.styles import ParagraphStyle
from datetime import datetime, timedelta
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import mm
from reportlab.lib.colors import blue
from reportlab.platypus import HRFlowable
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
import shutil
from google.colab import drive

from reportlab.lib.pagesizes import A4

In [30]:
# ======================
# LETTER TEMPLATES
# ======================
LETTER_TEXTS = {
    "Offered": """

On behalf of philanthropists Andrew and Kate Davis and the scholarship staff, I am pleased to offer you the 2026 Davis New Mexico Scholarship!  This wonderful opportunity is the gateway to achieving your dreams of an undergraduate education.

As you know, this scholarship will provide for the whole cost of attendance for all four years you are enrolled at one of the Davis New Mexico Scholarship partner colleges.

The Davis New Mexico Scholarship partner college(s) to which you have applied have already been notified of this decision.  However, there are next steps required for you to receive the scholarship:

[INSERT_BULLETS_HERE]

If, prior to April 27, you decide not to accept the scholarship, please contact the Davis New Mexico Scholarship immediately.  There are students on the waiting list who will become eligible should you decline.

Once again, congratulations on this life-changing achievement.  You have worked hard to put yourself in a position to receive this prestigious scholarship, and on behalf of Andrew and Kate Davis and all of us here at the Davis New Mexico Scholarship, we look forward to seeing your accomplishments in the future.

Now, go on and celebrate.

My most sincere congratulations,
""",
    "Denied": """On behalf of all of us here at the Davis New Mexico Scholarship program, this letter is to inform you that you have not been selected for the 2026 Davis New Mexico Scholarship.

Awarding the scholarship was an incredibly difficult decision, especially considering we had an unprecedented number of applicants. The fact that you completed the rigorous application process demonstrates your commitment to your educational future, and you have an undeniable head start in your college admissions process. Over the years, students who have applied for the Davis New Mexico Scholarship, regardless of whether or not they receive the scholarship, go on to college at a higher rate than their peers who did not apply.  I have no doubt you too will continue this trend.

We are certain that you will be successful in college next year, and we know that you will bring the same persistence and dedication to college that you have brought to this application. We at the scholarship wish you the best as you move forward in the college selection process.

If you have further questions, please do not hesitate to reach out to the college counselor you have been working with.

Sincerely,
""",
    "Waitlisted": """I am writing to inform you that you have been placed on the waitlist for the 2026 Davis New Mexico Scholarship. On behalf of the selection committee, I want you to know that awarding the scholarship was an incredibly difficult decision. I am certain that each one of you will be successful in college next year, and I know that you will bring the same persistence and dedication to college that you have brought to the scholarship application.

In order to remain eligible to receive the Davis New Mexico Scholarship, should you be selected in later selection rounds, please complete the following steps:

1). Please submit a transcript from the 2025 fall semester to the Davis New Mexico Scholarship, via your Overgrad Portal (details forthcoming via email). We must receive transcripts by January 31st, 2026.

2). Apply to Davis New Mexico Scholarship Partner College(s) by their Early Action and Regular Decision deadlines.

3). If you would like to apply to colleges other than our Partner Colleges, feel free to reach out to us to schedule a college counseling or essay editing session–we are happy to help you apply for admissions and financial aid.

4). Continue to check your mail and email, as the Davis New Mexico Scholarship selection committee will make another round of Davis New Mexico Scholarship selection for waitlisted students in January of 2026. Further selection rounds will follow as scholarships become available.

We will be able to notify you definitively whether or not you have been offered the Davis New Mexico Scholarship by April 28, 2026.  Should be selected off of the waitlist, it is imperative that you have kept up with the financial aid and admissions requirements for at least one of the Davis New Mexico Scholarship Partner Colleges.

Additionally, while no position off of the waiting list is guaranteed, I also want to remind you that ten students in last year’s Davis New Mexico Scholarship class were offered the scholarship off of the waitlist, so please stay dedicated to your college process. Of course, if you have further questions, please do not hesitate to reach out to us here at the scholarship or to the college counselor you have been working with.

Sincerely,
"""
}

In [26]:
# https://drive.google.com/file/d/1R0dCsTbON_Yoh3h0kc6h-_76y2MYZSwO/view?usp=sharing

drive.mount('/content/drive')
# file_id = "1R0dCsTbON_Yoh3h0kc6h-_76y2MYZSwO" #without advisory room number
file_id = "1R0dCsTbON_Yoh3h0kc6h-_76y2MYZSwO"

url = f"https://drive.google.com/uc?id={file_id}"
inputfile = "24-25 Drive Links + Mail Merge - Student Letters.csv"             # your CSV file
gdown.download(url, inputfile, quiet=False)

INPUT_CSV = inputfile
# output_folder = "open_house_pdfs"        # folder to save PDFs
# output_folder = "/content/drive/My Drive/open_house_pdfs"
OUTPUT_FOLDER = "/content/drive/My Drive/DavisPDFs"
LOGO_FILE = "/content/drive/My Drive/Logo.png"               # your logo file (PNG)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

signature_file = "/content/drive/My Drive/sig.png"
LOGO_FILE = "/content/drive/My Drive/Logo.png"
LOGO_CROPPED = "/content/drive/My Drive/croppedLogo.png"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Downloading...
From: https://drive.google.com/uc?id=1R0dCsTbON_Yoh3h0kc6h-_76y2MYZSwO
To: /content/24-25 Drive Links + Mail Merge - Student Letters.csv
100%|██████████| 17.5k/17.5k [00:00<00:00, 27.5MB/s]


In [27]:
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

# Assuming you have Delicious-Bold.ttf in your project folder
pdfmetrics.registerFont(TTFont("Delicious-Bold", "/content/drive/My Drive/Delicious-Bold.ttf"))

In [28]:
from PIL import Image as PILImage

# Open the signature image
img = PILImage.open(signature_file).convert("RGBA")  # Ensure RGBA mode

# Make white (or any color) transparent
datas = img.getdata()
newData = []
for item in datas:
    # item = (R, G, B, A)
    if item[0] > 240 and item[1] > 240 and item[2] > 240:  # white threshold
        newData.append((255, 255, 255, 0))  # transparent
    else:
        newData.append(item)
img.putdata(newData)

# Save a temporary PNG with transparency
temp_file = "signature_transparent.png"
img.save(temp_file, "PNG")

# Now use ReportLab Image
from reportlab.platypus import Image


In [31]:
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Frame, PageTemplate
from reportlab.lib.pagesizes import LETTER
STRIPE_WIDTH = 144
# Remove old folder if it exists, then create a new one
if os.path.exists(OUTPUT_FOLDER):
    shutil.rmtree(OUTPUT_FOLDER)
os.makedirs(OUTPUT_FOLDER)

# === Styles ===
styles = getSampleStyleSheet()

# Left-aligned text (body of letter)
style_left = ParagraphStyle(
    'left',
    parent=styles['Normal'],
    fontName="Delicious-Bold",  # registered font name
    fontSize=12,       # reduced from 11
    leading=13,        # reduced from 16
    alignment=TA_LEFT,
    spaceAfter=2       # reduced from 8
)

# Centered bold text (e.g., "Congratulations!")
style_center_bold = ParagraphStyle(
    'center_bold',
    parent=styles['Normal'],
    fontName="Delicious-Bold",  # registered font name
    fontSize=16,       # reduced from 18
    leading=16,        # reduced from 22
    alignment=TA_CENTER,
    spaceAfter=5,      # reduced from 12
)

style_bullet = ParagraphStyle(
    'BulletIndented',
    parent=styles['Normal'],
    fontName="Delicious-Bold",  # registered font name
    fontSize=11,
    leftIndent=24,
    firstLineIndent=-12,
    spaceAfter=6,
)

# === Function to build each PDF ===
def build_letter_pdf(pdf_file, full_name, status):
    width, height = LETTER
    doc = SimpleDocTemplate(pdf_file, pagesize=LETTER,
                            leftMargin=30, rightMargin=STRIPE_WIDTH + 20,
                            topMargin=20, bottomMargin=30)

    # Platypus elements
    elements = []
    today = datetime.today().strftime("%B %d, %Y")
    elements.append(Paragraph(today, style_left))
    elements.append(Spacer(1, 24))

    if status.lower() == "offered":
        elements.append(Paragraph("Dear "+full_name+",", style_left))
        elements.append(Spacer(1, 12))
        elements.append(Paragraph("Congratulations!", style_center_bold))
        elements.append(Spacer(1, 12))

        offered_text = LETTER_TEXTS["Offered"]

        # Split where placeholder appears
        before, after = offered_text.split("[INSERT_BULLETS_HERE]")

        # --- BEFORE — preserve multiple paragraphs ---
        for para in before.strip().split("\n\n"):
            elements.append(Paragraph(para.strip(), style_left))
            elements.append(Spacer(1, 10))

        # Bullet list content
        bullet_points = [
            "You must complete all of the requirements for graduation from your high school and the entrance requirements for your college or university, including working with the Davis New Mexico Scholarship office to attend applicable orientations.",
            "Attend the Davis New Mexico Scholarship Welcome Lunch at 12:00pm on Thursday, December 18th. The event will be at the Scottish Rite Temple in Santa Fe. An invitation will follow via Paperless Post.",
            "By April 27, 2026, you must notify the Davis New Mexico Scholarship office if you will be attending a Davis New Mexico partner college by scheduling a meeting with your parents to sign the attached contract and formally accept the scholarship.  Please send an email to Alejandra Palos at apalos@davisnm.org to schedule the meeting.",
            "You must attend the 2026 Davis New Mexico Scholarship Celebration on May 14, 2026, at the Club at Las Campanas in Santa Fe. Details and a formal invitation will be forthcoming.",
        ]

        # Add bullets with indentation
        for i, point in enumerate(bullet_points, 1):
            elements.append(Paragraph(f"{i}). {point}", style_bullet))
            elements.append(Spacer(1, 4))

        # --- AFTER — preserve multiple paragraphs ---
        for para in after.strip().split("\n\n"):
            elements.append(Spacer(1, 10))
            elements.append(Paragraph(para.strip(), style_left))
            elements.append(Spacer(1, 10))
    else:
      elements.append(Paragraph("Dear "+full_name+",", style_left))
      elements.append(Spacer(1, 12))
      # All non-offered statuses use plain text
      for para in LETTER_TEXTS[status].strip().split("\n\n"):
            elements.append(Spacer(1, 10))
            elements.append(Paragraph(para.strip(), style_left))
            elements.append(Spacer(1, 10))


    def draw_stripe(canvas, doc):
        width, height = LETTER

        # Red stripe on right
        canvas.setFillColor(colors.darkred)
        canvas.rect(width - STRIPE_WIDTH, 0, STRIPE_WIDTH, height, fill=1, stroke=0)

        # Logo on stripe: always drawn on every page
        if os.path.exists(LOGO_FILE):
            logo_w, logo_h = 80, 80
            logo_x = width - STRIPE_WIDTH/2 - logo_w/2
            logo_y = height/2
            canvas.drawImage(LOGO_FILE, logo_x, logo_y, width=logo_w, height=logo_h, mask='auto')

            # Contact info below logo
            canvas.setFillColor(colors.white)
            canvas.setFont("Helvetica-Bold", 10)
            contact_lines = ["505.982.8921", "www.davisnm.org", "1101 Camino de Cruz Blanca", "Santa Fe, NM 87505"]
            for i, line in enumerate(contact_lines):
                text_y = logo_y - 15 - i*12
                canvas.drawCentredString(width - STRIPE_WIDTH/2, text_y, line)

        # Faded background logo: only on first page
        if canvas.getPageNumber() == 1 and os.path.exists(LOGO_CROPPED):
            canvas.saveState()

            # Make it semi-transparent (optional)
            canvas.setFillAlpha(0.05)

            # Desired width and height of the logo
            logo_width  = 400  # adjust as needed
            logo_height = 400  # adjust as needed

            # Position at lower-left corner
            x_pos = 0
            y_pos = 0

            # Draw the image
            canvas.drawImage(LOGO_CROPPED, x_pos, y_pos, width=logo_width, height=logo_height, mask='auto')

            canvas.restoreState()
        signature = Image(temp_file)
        signature.drawHeight = 40  # adjust as needed
        signature.drawWidth = 120
        signature.hAlign = 'LEFT'
        elements.append(Spacer(1, 12))
        elements.append(signature)
        elements.append(Spacer(1, 6))

        # Add printed name and title
        elements.append(Paragraph("Sam Ritter", style_left))
        elements.append(Paragraph("Director, Davis New Mexico Scholarship", style_left))



    # Frame left of stripe
    frame = Frame(doc.leftMargin, doc.bottomMargin,
                  doc.width, doc.height,
                  leftPadding=0, rightPadding=0, topPadding=0, bottomPadding=0)
    template = PageTemplate(id='letter', frames=[frame], onPage=draw_stripe)
    doc.addPageTemplates([template])

    doc.build(elements)

with open(INPUT_CSV, newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        file_name = row.get("File Name", "Student").strip()  # Use File Name column
        full_name = row.get("Full Name", "Student").strip()
        status = row.get("Status", "Offered").strip().capitalize()

        # Ensure the PDF filename is safe (no illegal characters)
        safe_file_name = "".join(c for c in file_name if c.isalnum() or c in (" ", "_", "-")).rstrip()
        pdf_file = os.path.join(OUTPUT_FOLDER, f"{safe_file_name}.pdf")

        build_letter_pdf(pdf_file, full_name, status)
        print(f"✅ Generated PDF for {full_name} ({status}) as {safe_file_name}.pdf")


✅ Generated PDF for Tristan Abeyta (Waitlisted) as 47269146_Tristan Abeyta.pdf
✅ Generated PDF for Geraldine Acosta Morales (Denied) as 45103907_Geraldine Acosta Morales.pdf
✅ Generated PDF for Naomi Aguilar (Denied) as 44336975_Naomi Aguilar.pdf
✅ Generated PDF for Emily Aguirre (Offered) as 44559245_Emily Aguirre.pdf
✅ Generated PDF for Manuel Aguirre (Offered) as 47175739_Manuel Aguirre.pdf
✅ Generated PDF for Jazliyah Ahiyite (Denied) as 39726172_Jazliyah Ahiyite.pdf
✅ Generated PDF for Abraham Alejos (Offered) as 44362492_Abraham Alejos.pdf
✅ Generated PDF for Alexa Alfonso Medina (Offered) as 44942860_Alexa Alfonso Medina.pdf
✅ Generated PDF for Daniela Almanza (Denied) as 47395277_Daniela Almanza.pdf
✅ Generated PDF for Daphne Almeida (Denied) as 47298870_Daphne Almeida.pdf
✅ Generated PDF for Vanessa Almeida (Denied) as 47396881_Vanessa Almeida.pdf
✅ Generated PDF for Rebeca Alvarez (Denied) as 48147390_Rebeca Alvarez.pdf
✅ Generated PDF for Adamary Amador (Offered) as 47308285

In [101]:
# ======================
# LETTER TEMPLATES
# ======================
LETTER_TEXTS_0 = {
    "Offered": """

On behalf of philanthropist Andrew Davis and the scholarship staff, I am pleased to offer you the 2026 Davis New Mexico Scholarship! This wonderful opportunity is the gateway to achieving your dreams of an undergraduate education.

As you know, this scholarship will provide for the whole cost of attendance for all four years you are enrolled at one of the Davis New Mexico Scholarship partner colleges.

The Davis New Mexico Scholarship partner college(s) to which you have applied have already been notified of this decision. However, there are next steps required for you to receive the scholarship:

1). You must complete all of the requirements for graduation from your high school and the entrance requirements for your college or university, including working with the Davis New Mexico Scholarship office to attend applicable orientations.

2). Please strongly consider attending our Davis New Mexico Scholarship Welcome Lunch on December 18, 2024. The event will take place from 12:00-2:00 at the Scottish Rite Temple in Santa Fe, NM. A formal invitation will be sent via email.

3). Please submit a transcript from the 2025 fall semester to the Davis New Mexico Scholarship, via your Overgrad Portal (details forthcoming via email). We must receive transcripts by January 30, 2026.

4). By April 27, 2026, you must notify the Davis New Mexico Scholarship office if you will be attending a Davis New Mexico partner college by scheduling a meeting with your parents to sign the attached contract and formally accept the scholarship.  Please send an email to Alejandra Palos at apalos@davisnm.org to schedule the meeting.

5). You must attend the 2026 Davis New Mexico Scholarship Celebration on May 14, 2025, at the Club at Las Campanas in Santa Fe. Details and a formal invitation will be forthcoming.


If, prior to April 27, you decide not to accept the scholarship, please contact the Davis New Mexico Scholarship immediately. There are students on the waiting list who will become eligible should you decline.

Once again, congratulations on this life-changing achievement. You have worked hard to put yourself in a position to receive this prestigious scholarship, and on behalf of Andrew Davis and all of us here at the Davis New Mexico Scholarship, we look forward to seeing your accomplishments in the future.

My most sincere congratulations,
""",
    "Denied": """On behalf of the Shelby Cullom Davis Charitable Fund, this letter is to inform you that you have not been selected for the 2026 Davis New Mexico Scholarship.

Awarding the scholarship was an incredibly difficult decision, especially considering we had an unprecedented number of applicants. The fact that you completed the rigorous application process demonstrates your commitment to your educational future, and you have an undeniable head start in your college admissions process. Over the years, students who have applied for the Davis New Mexico Scholarship, regardless of whether or not they receive the scholarship, go on to college at a higher rate than their peers who did not apply. I have no doubt you too will continue this trend.

We are certain that you will be successful in college next year, and we know that you will bring the same persistence and dedication to college that you have brought to this application. We at the scholarship wish you the best as you move forward in the college selection process.

If you have further questions, please do not hesitate to reach out to the college counselor you have been working with.

Sincerely,
""",
    "Waitlist": """I am writing to inform you that you have been placed on the waitlist for the 2026 Davis New Mexico Scholarship. On behalf of the selection committee, I want you to know that awarding the scholarship was an incredibly difficult decision. I am certain that each one of you will be successful in college next year, and I know that you will bring the same persistence and dedication to college that you have brought to the scholarship application.

In order to remain eligible to receive the Davis New Mexico Scholarship, should you be selected in later selection rounds, please complete the following steps:

Please submit a transcript from the 2025 fall semester to the Davis New Mexico Scholarship, via your Overgrad Portal (details forthcoming via email). We must receive transcripts by January 30th, 2025.
Apply to Davis New Mexico Scholarship Partner College(s) by their Early Action and Regular Decision deadlines.
If you would like to apply to colleges other than our Partner Colleges, feel free to reach out to us to schedule a college counseling or essay editing session–we are happy to help you apply for admissions and financial aid.
Continue to check your mail and email, as the Davis New Mexico Scholarship selection committee will make another round of Davis New Mexico Scholarship selection for waitlisted students in January of 2026. Further selection rounds will follow as scholarships become available.

We will be able to notify you definitively whether or not you have been offered the Davis New Mexico Scholarship by April 30, 2026. Should be selected off of the waitlist, it is imperative that you have kept up with the financial aid and admissions requirements for at least one of the Davis New Mexico Scholarship Partner Colleges.

Additionally, while no position off of the waiting list is guaranteed, I also want to remind you that ten students in last year’s Davis New Mexico Scholarship class were offered the scholarship off of the waitlist, so please stay dedicated to your college process. Of course, if you have further questions, please do not hesitate to reach out to us here at the scholarship or to the college counselor you have been working with.

Sincerely,
"""
}

In [102]:
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Frame, PageTemplate
from reportlab.lib.pagesizes import LETTER
STRIPE_WIDTH = 144
# Remove old folder if it exists, then create a new one
if os.path.exists(OUTPUT_FOLDER):
    shutil.rmtree(OUTPUT_FOLDER)
os.makedirs(OUTPUT_FOLDER)

# === Styles ===
styles = getSampleStyleSheet()

# Left-aligned text (body of letter)
style_left = ParagraphStyle(
    'left',
    parent=styles['Normal'],
    fontName="Delicious-Bold",  # registered font name
    fontSize=11,       # reduced from 11
    leading=13,        # reduced from 16
    alignment=TA_LEFT,
    spaceAfter=2       # reduced from 8
)

# Centered bold text (e.g., "Congratulations!")
style_center_bold = ParagraphStyle(
    'center_bold',
    parent=styles['Normal'],
    fontName="Delicious-Bold",  # registered font name
    fontSize=16,       # reduced from 18
    leading=16,        # reduced from 22
    alignment=TA_CENTER,
    spaceAfter=5,      # reduced from 12
)

# === Function to build each PDF ===
def build_letter_pdf(pdf_file, full_name, status):
    width, height = LETTER
    doc = SimpleDocTemplate(pdf_file, pagesize=LETTER,
                            leftMargin=30, rightMargin=STRIPE_WIDTH + 20,
                            topMargin=20, bottomMargin=30)

    # Platypus elements
    elements = []
    today = datetime.today().strftime("%B %d, %Y")
    elements.append(Paragraph(today, style_left))
    elements.append(Spacer(1, 24))

    if status.lower() == "offered":
        elements.append(Paragraph("Dear,", style_left))
        elements.append(Spacer(1, 12))
        elements.append(Paragraph("Congratulations!", style_center_bold))
        elements.append(Spacer(1, 12))

    text = LETTER_TEXTS_0.get(status.capitalize(), LETTER_TEXTS_0["Offered"])
    for paragraph in text.strip().split("\n\n"):
        elements.append(Paragraph(paragraph.strip(), style_left))
        elements.append(Spacer(1, 12))

    def draw_stripe(canvas, doc):
        width, height = LETTER

        # Red stripe on right
        canvas.setFillColor(colors.darkred)
        canvas.rect(width - STRIPE_WIDTH, 0, STRIPE_WIDTH, height, fill=1, stroke=0)

        # Logo on stripe: always drawn on every page
        if os.path.exists(LOGO_FILE):
            logo_w, logo_h = 80, 80
            logo_x = width - STRIPE_WIDTH/2 - logo_w/2
            logo_y = height/2
            canvas.drawImage(LOGO_FILE, logo_x, logo_y, width=logo_w, height=logo_h, mask='auto')

            # Contact info below logo
            canvas.setFillColor(colors.white)
            canvas.setFont("Helvetica-Bold", 10)
            contact_lines = ["505.982.8921", "www.davisnm.org", "1101 Camino de Cruz Blanca", "Santa Fe, NM 87505"]
            for i, line in enumerate(contact_lines):
                text_y = logo_y - 15 - i*12
                canvas.drawCentredString(width - STRIPE_WIDTH/2, text_y, line)

        # Faded background logo: only on first page
        # Faded background logo: only on first page
        if canvas.getPageNumber() == 1 and os.path.exists(LOGO_CROPPED):
            canvas.saveState()

            # Make it semi-transparent (optional)
            canvas.setFillAlpha(0.05)

            # Desired width and height of the logo
            logo_width  = 400  # adjust as needed
            logo_height = 400  # adjust as needed

            # Position at lower-left corner
            x_pos = 0
            y_pos = 0

            # Draw the image
            canvas.drawImage(LOGO_CROPPED, x_pos, y_pos, width=logo_width, height=logo_height, mask='auto')

            canvas.restoreState()
        signature = Image(temp_file)
        signature.drawHeight = 40  # adjust as needed
        signature.drawWidth = 120
        signature.hAlign = 'LEFT'
        elements.append(Spacer(1, 12))
        elements.append(signature)
        elements.append(Spacer(1, 6))

        # Add printed name and title
        elements.append(Paragraph("Sam Ritter", style_left))
        elements.append(Paragraph("Director, Davis New Mexico Scholarship", style_left))



    # Frame left of stripe
    frame = Frame(doc.leftMargin, doc.bottomMargin,
                  doc.width, doc.height,
                  leftPadding=0, rightPadding=0, topPadding=0, bottomPadding=0)
    template = PageTemplate(id='letter', frames=[frame], onPage=draw_stripe)
    doc.addPageTemplates([template])

    doc.build(elements)

with open(INPUT_CSV, newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        file_name = row.get("File Name", "Student").strip()  # Use File Name column
        full_name = row.get("Full Name", "Student").strip()
        status = row.get("Status", "Offered").strip().capitalize()

        # Ensure the PDF filename is safe (no illegal characters)
        safe_file_name = "".join(c for c in file_name if c.isalnum() or c in (" ", "_", "-")).rstrip()
        pdf_file = os.path.join(OUTPUT_FOLDER, f"{safe_file_name}.pdf")

        build_letter_pdf(pdf_file, full_name, status)
        print(f"✅ Generated PDF for {full_name} ({status}) as {safe_file_name}.pdf")


✅ Generated PDF for Shiloh Conn (Waitlist) as 38469672_Shiloh Conn.pdf
✅ Generated PDF for Kitana James (Waitlist) as 39195692_Kitana James.pdf
✅ Generated PDF for Fernanda Rangel (Waitlist) as 40034130_Fernanda Rangel.pdf
✅ Generated PDF for Jazmin Zermeno (Waitlist) as 40452024_Jazmin Zermeno.pdf
✅ Generated PDF for Ana Chavez (Waitlist) as 40452036_Ana Chavez.pdf
✅ Generated PDF for Sakeena Nguyen (Waitlist) as 40575940_Sakeena Nguyen.pdf
✅ Generated PDF for Celine Villalobos (Waitlist) as 40836532_Celine Villalobos.pdf
✅ Generated PDF for Giselle Diaz (Waitlist) as 40842829_Giselle Diaz.pdf
✅ Generated PDF for Keila Aguirre Jaquez (Waitlist) as 40883743_Keila Aguirre Jaquez.pdf
✅ Generated PDF for Mariel Orozco-Aguilera (Waitlist) as 40944426_Dora Orozco-Aguilera.pdf
✅ Generated PDF for Madison Sandoval (Waitlist) as 40944429_Madison Sandoval.pdf
✅ Generated PDF for Amelia Suina (Waitlist) as 40944560_Amelia Suina.pdf
✅ Generated PDF for Ana Sagché (Waitlist) as 42231961_Ana Sagché